In [8]:
# code taken and refined from: https://github.com/osh/KerasGAN
# check out https://github.com/soumith/ganhacks for tips tricks on training gans

import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import os
from IPython import display
import tqdm
from sklearn.metrics import accuracy_score
os.environ["KERAS_BACKEND"] = "tensorflow"
%matplotlib inline

from keras.datasets import mnist
from keras.utils.np_utils import to_categorical

# Load pre-shuffled MNIST data into train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print("Training input shape: ", X_train.shape)
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1) #tf ordering, depth/channels at last index
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)


X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train -= 127.5
X_train /= 127.5
X_test -= 127.5
X_test /= 127.5 # scaling to [-1,1] matching tanh output of generator

print("Training input shape reshaped: ",X_train.shape)
#labels_train = to_categorical(y_train)
#labels_test = to_categorical(y_test)
#print("Training labels reshaped: ", labels_train.shape)

/home/timomoeller/.virtualenvs/keras/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


Training input shape:  (60000, 28, 28)
Training input shape reshaped:  (60000, 28, 28, 1)


In [1]:
from keras import backend as K
from keras.layers import Input
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.core import Activation, Dense, Dropout, Flatten, Reshape
from keras.layers.convolutional import Convolution2D, UpSampling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

from keras.models import Model

# def make_trainable(net,  val):
#     net.trainable = val
#     for l in net.layers:
#         l.trainable = val

Using TensorFlow backend.


In [3]:
#Generative model

nch = 200

g_opt = Adam(lr=2e-4,beta_1=0.5)
g_inp = Input(shape=(100,))

H = Dense(nch*14*14,init='glorot_normal',input_shape=(100,))(g_inp)
H = LeakyReLU(alpha=0.2)(H)
H = Reshape((14,14,nch))(H)
H = BatchNormalization(mode=2,axis=-1)(H) # if using theano change axis argument here!!!
H = UpSampling2D(size=(2,2))(H)
H = Convolution2D(int(nch/2),3,3,border_mode='same',init='glorot_normal')(H)
H = BatchNormalization(mode=2,axis=-1)(H)
H = LeakyReLU(alpha=0.2)(H)
H = Dropout(0.25)(H)
H = Convolution2D(int(nch/4),3,3,border_mode='same',init='glorot_normal')(H)
H = BatchNormalization(mode=2)(H)
H = LeakyReLU(alpha=0.2)(H)
H = Dropout(0.25)(H)

#H = Convolution2D(1, 1, 1, border_mode='same', init='glorot_uniform')(H)
#g_pred = Activation('sigmoid')(H)
g_pred = Dense(1,init='glorot_normal',activation='tanh')(H)

generator = Model(g_inp,g_pred)
generator.compile(loss='binary_crossentropy',optimizer=g_opt)
#generator.summary()

In [4]:
# discriminator
d_opt = Adam(lr=2e-4,beta_1=0.5)
d_inp = Input(shape=X_train.shape[1:])

H = Convolution2D(256,5,5,subsample=(2,2),border_mode='same',
                  init='glorot_normal',input_shape=X_train.shape[1:])(d_inp)
#H = BatchNormalization(mode=2,axis=-1)(H)
H = LeakyReLU(alpha=0.2)(H)
#H = AveragePooling2D(pool_size=(2,2))(H)
H = Dropout(0.25)(H)

H = Convolution2D(512,5,5,subsample=(2,2),border_mode='same',init='glorot_normal')(H)
#H = BatchNormalization(mode=2,axis=-1)(H)
H = LeakyReLU(alpha=0.2)(H)
#H = AveragePooling2D(pool_size=(2,2))(H)
H = Dropout(0.25)(H)

H = Flatten()(H)
H = Dense(256)(H)
#H = BatchNormalization(mode=2,axis=-1)(H) # without BN here BN in prev layers makes train impossible
H = LeakyReLU(alpha=0.2)(H)
H = Dropout(0.25)(H)
d_pred = Dense(1,activation='sigmoid')(H)

discriminator = Model(d_inp,d_pred)
discriminator.compile(loss='binary_crossentropy',optimizer = d_opt)#,metrics=['accuracy'])
#discriminator.summary()

In [15]:
# GAN, put together
#make_trainable(discriminator,False)
gan_inp = Input(shape=(100,))
H = generator(gan_inp)
gan_pred = discriminator(H)
GAN = Model(gan_inp,gan_pred)
GAN.compile(loss='binary_crossentropy', optimizer = g_opt)
GAN.summary()

# set up loss storage vector
losses = {"d":[], "g":[]}

#helper fcts
def plot_loss(losses):
        display.clear_output(wait=True)
        display.display(plt.gcf())
        plt.figure(figsize=(12,6))
        plt.plot(losses["d"], label='discriminitive loss')
        #plt.plot(losses["d"][1], label='discriminitive acc')
        plt.plot(losses["g"], label='generative loss')
        plt.legend()
        plt.show()
        

def plot_gen(n_ex=16,dim=(2,8), figsize=(10,3) ):
    noise = np.random.uniform(-1,1,size=[n_ex,100])
    generated_images = generator.predict(noise)

    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0],dim[1],i+1)
        img = generated_images[i,:,:,0]
        plt.imshow(img)
        plt.axis('off')
    plt.tight_layout()
    plt.show()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_13 (InputLayer)            (None, 100)           0                                            
____________________________________________________________________________________________________
model_5 (Model)                  (None, 28, 28, 1)     540401      input_13[0][0]                   
____________________________________________________________________________________________________
model_9 (Model)                  (None, 1)             48451       model_5[1][0]                    
Total params: 588,852
Trainable params: 588,352
Non-trainable params: 500
____________________________________________________________________________________________________


In [6]:
##### pre train discriminator
num_samples= 10000
random_g_inp = np.random.normal(0,1,size=(num_samples,100))
random_gen = generator.predict(random_g_inp)

idxX = np.random.permutation(X_train.shape[0])[:num_samples]
X = np.concatenate((X_train[idxX] ,random_gen))
y = np.zeros((X.shape[0],1))
y[:num_samples,0]=1

idx = np.random.permutation(num_samples*2)
print(str(X.shape) + ' labels: ' + str(y.shape))
discriminator.fit(X[idx],y[idx],nb_epoch=1,batch_size=32)


num_samples= 2000
random_g_inp = np.random.normal(0,1,size=(num_samples,100))
random_gen = generator.predict(random_g_inp)

idxX = np.random.permutation(X_train.shape[0])[:num_samples]
X = np.concatenate((X_train[idxX] ,random_gen))
y = np.zeros((X.shape[0],1))
y[:num_samples,0]=1.0

yhat = discriminator.predict(X)
np.sum(np.abs(y-yhat))#/(2*num_samples)


(20000, 28, 28, 1) labels: (20000, 1)
Epoch 1/1
20000/20000 [==============================] - 57s - loss: 0.0022    


In [17]:
# set up loss storage vector
losses = {"d":[], "g":[]}

array([[ 1.],
       [ 1.],
       [ 1.],
       ..., 
       [ 1.],
       [ 1.],
       [ 1.]], dtype=float32)

In [6]:
def get_gradients(model):
    """Return the gradient of every trainable weight in model

    Parameters
    -----------
    model : a keras model instance

    First, find all tensors which are trainable in the model. Surprisingly,
    `model.trainable_weights` will return tensors for which
    trainable=False has been set on their layer (last time I checked), hence the extra check.
    Next, get the gradients of the loss with respect to the weights.

    """
    #weights = [tensor for tensor in model.trainable_weights if model.get_layer(tensor.name[:-2]).trainable]
    weights = model.trainable_weights
    optimizer = model.optimizer

    return optimizer.get_gradients(model.total_loss, weights)

In [13]:
def train_for_n(nb_epoch=5000, plt_frq=25,BATCH_SIZE=32):

    for e in tqdm.trange(nb_epoch):  
        
        batch_d = BATCH_SIZE
        # Make generative images
        idx = np.random.randint(0,X_train.shape[0],size=int(batch_d/2))
        image_batch = X_train[idx,:,:,:]    
        noise_gen = np.random.normal(0,1,size=[int(batch_d/2),100])
        generated_images = generator.predict(noise_gen)
        
        # Train discriminator on real images 
        discriminator.trainable = True
        y1 = np.ones((int(batch_d/2),1)) + np.random.uniform(-0.2,0.3,size=(int(batch_d/2),1))
        d_loss  = discriminator.train_on_batch(image_batch,y1)
        #losses["d"].append(d_loss)
        # and on generated images
        y12 = np.zeros((int(batch_d/2),1)) + np.random.uniform(0,0.2,size=(int(batch_d/2),1))
        d_loss2  = discriminator.train_on_batch(generated_images,y12)
        losses["d"].append((d_loss+d_loss2)/2)
    
    
        batch_g = BATCH_SIZE
        #train generator on 2 batches
        #discriminator.trainable(False)
        discriminator.trainable = False
        noise_tr = np.random.normal(0,1,size=[batch_d,100])
        y2 = np.ones((batch_d,1))  + np.random.uniform(-0.2,0.3,size=(batch_g,1)) # generator wants to create real labels
        g_loss = GAN.train_on_batch(noise_tr, y2)
        losses["g"].append(g_loss)
        
        # Updates plots
        if e%plt_frq==plt_frq-1:
            discriminator.trainable = True
            gradients = get_gradients(GAN)
            
            print(len(gradients))
            sess = K.get_session()
            with sess.as_default():
                print(gradients[0].eval())
            
            #plot_loss(losses)
            #plot_gen()

In [14]:
train_for_n(nb_epoch=5, plt_frq=1, BATCH_SIZE=50)

  0%|          | 0/5 [00:00<?, ?it/s]

22


InvalidArgumentError: You must feed a value for placeholder tensor 'keras_learning_phase' with dtype bool
	 [[Node: keras_learning_phase = Placeholder[dtype=DT_BOOL, shape=[], _device="/job:localhost/replica:0/task:0/gpu:0"]()]]
	 [[Node: moments_3/sufficient_statistics/Shape/_195 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_357_moments_3/sufficient_statistics/Shape", tensor_type=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'keras_learning_phase', defined at:
  File "/home/timomoeller/anaconda3/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/timomoeller/anaconda3/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/timomoeller/.virtualenvs/keras/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_ne

In [14]:

d_opt.lr.assign(1e-4)
g_opt.lr.assign(1e-4)


<tf.Tensor 'Assign_181:0' shape=() dtype=float32_ref>